In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from sklearn.metrics import f1_score

# Configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch_size = 64
epochs = 25

# Data Preparation
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# LeNet Model
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16*4*4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.max_pool2d(x, 2, 2)
        x = torch.relu(self.conv2(x))
        x = torch.max_pool2d(x, 2, 2)
        x = x.view(-1, 16*4*4)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Initialize Model
model = LeNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

# Training Loop
def train_model(model, loader):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(loader)}")

# Evaluation
def evaluate_model(model, loader):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    return all_labels, all_preds

train_model(model, train_loader)
true_labels, predicted_labels = evaluate_model(model, test_loader)
accuracy = sum([1 for true, pred in zip(true_labels, predicted_labels) if true == pred]) / len(true_labels)
f1 = f1_score(true_labels, predicted_labels, average='weighted')

print(f"Accuracy: {accuracy*100:.2f}%, F1 Score: {f1:.2f}")


Epoch 1/25, Loss: 0.2458650893730316
Epoch 2/25, Loss: 0.07202167023044787
Epoch 3/25, Loss: 0.05244105090038416
Epoch 4/25, Loss: 0.04148007498296754
Epoch 5/25, Loss: 0.03419133901830265
Epoch 6/25, Loss: 0.02942045467729712
Epoch 7/25, Loss: 0.024943765408990196
Epoch 8/25, Loss: 0.02281281545871405
Epoch 9/25, Loss: 0.019312328867231575
Epoch 10/25, Loss: 0.016814906713857464
Epoch 11/25, Loss: 0.015062828617703847
Epoch 12/25, Loss: 0.015372558772077624
Epoch 13/25, Loss: 0.012928580608654566
Epoch 14/25, Loss: 0.01025229639353924
Epoch 15/25, Loss: 0.010797202584284423
Epoch 16/25, Loss: 0.010401211253421647
Epoch 17/25, Loss: 0.007946938997027872
Epoch 18/25, Loss: 0.008747007918003464
Epoch 19/25, Loss: 0.009062762238956443
Epoch 20/25, Loss: 0.007707487706723389
Epoch 21/25, Loss: 0.007038458869113373
Epoch 22/25, Loss: 0.007218020430296681
Epoch 23/25, Loss: 0.008689647627528274
Epoch 24/25, Loss: 0.004622160426381923
Epoch 25/25, Loss: 0.007249597535165571
Accuracy: 99.00%, 

In [3]:
# Function to Train and Evaluate with Hyperparameter Logging
def train_and_evaluate(model, train_loader, test_loader, lr, batch_size, optimizer_type):
    print(f"Training with LR={lr}, Batch Size={batch_size}, Optimizer={optimizer_type.__name__}")
    
    # Reinitialize Model
    model = LeNet().to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optimizer_type(model.parameters(), lr=lr)
    
    # Training Loop
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader)}")
    
    # Evaluation
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    accuracy = sum([1 for true, pred in zip(all_labels, all_preds) if true == pred]) / len(all_labels)
    f1 = f1_score(all_labels, all_preds, average='weighted')

    print(f"Accuracy: {accuracy*100:.2f}%, F1 Score: {f1:.2f}")
    return accuracy, f1

# Hyperparameter Tuning Loop
learning_rates = [0.01, 0.001, 0.0001]
batch_sizes = [32, 64, 128]
optimizers = [optim.Adam, optim.SGD, optim.RMSprop]

results = []

for lr in learning_rates:
    for batch_size in batch_sizes:
        for opt in optimizers:
            # Update DataLoader for Batch Size
            train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
            test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
            
            acc, f1 = train_and_evaluate(LeNet(), train_loader, test_loader, lr, batch_size, opt)
            results.append({'lr': lr, 'batch_size': batch_size, 'optimizer': opt.__name__, 'accuracy': acc, 'f1_score': f1})


Training with LR=0.01, Batch Size=32, Optimizer=Adam
Epoch 1/25, Loss: 0.5985796714087327
Epoch 2/25, Loss: 0.3982830466757218
Epoch 3/25, Loss: 0.370960235658288
Epoch 4/25, Loss: 0.35840828212797643
Epoch 5/25, Loss: 0.3498827578149736
Epoch 6/25, Loss: 0.33676523219632604
Epoch 7/25, Loss: 0.341860643829902
Epoch 8/25, Loss: 0.3411294964564343
Epoch 9/25, Loss: 0.3348617584099372
Epoch 10/25, Loss: 0.3275609507886072
Epoch 11/25, Loss: 0.33790144894917806
Epoch 12/25, Loss: 0.3175504847895354
Epoch 13/25, Loss: 0.3288730463615308
Epoch 14/25, Loss: 0.3241818760889272
Epoch 15/25, Loss: 0.3511347201017042
Epoch 16/25, Loss: 0.32416921319067477
Epoch 17/25, Loss: 0.342156790695712
Epoch 18/25, Loss: 0.33454803063198923
Epoch 19/25, Loss: 0.31661266812359296
Epoch 20/25, Loss: 0.363572636140883
Epoch 21/25, Loss: 0.32621568090158204
Epoch 22/25, Loss: 0.33905524789094926
Epoch 23/25, Loss: 0.3194112988010049
Epoch 24/25, Loss: 0.32206148709729315
Epoch 25/25, Loss: 0.3562324930942307
A

Epoch 24/25, Loss: 0.158036047493471
Epoch 25/25, Loss: 0.14758261135900452
Accuracy: 97.31%, F1 Score: 0.97
Training with LR=0.01, Batch Size=128, Optimizer=RMSprop
Epoch 1/25, Loss: 1.8132115495738699
Epoch 2/25, Loss: 0.8735669227297118
Epoch 3/25, Loss: 0.7700469423331686
Epoch 4/25, Loss: 0.7144869037274358
Epoch 5/25, Loss: 0.6760384763863041
Epoch 6/25, Loss: 0.65965937538696
Epoch 7/25, Loss: 0.6066192260810307
Epoch 8/25, Loss: 0.47522686882568066
Epoch 9/25, Loss: 0.4126398598969872
Epoch 10/25, Loss: 0.3883197374308287
Epoch 11/25, Loss: 0.3626853012835293
Epoch 12/25, Loss: 0.3550148580247151
Epoch 13/25, Loss: 0.3421150600509857
Epoch 14/25, Loss: 0.3289591173119128
Epoch 15/25, Loss: 0.3347303142933957
Epoch 16/25, Loss: 0.323472504152545
Epoch 17/25, Loss: 0.31485034662014894
Epoch 18/25, Loss: 0.31873028067701153
Epoch 19/25, Loss: 0.31190337288354253
Epoch 20/25, Loss: 0.3106046061811925
Epoch 21/25, Loss: 0.31033643646471537
Epoch 22/25, Loss: 0.29696985486664496
Epoc

Epoch 21/25, Loss: 0.07510646884994847
Epoch 22/25, Loss: 0.07238130100063488
Epoch 23/25, Loss: 0.07104277156039214
Epoch 24/25, Loss: 0.06822486075439624
Epoch 25/25, Loss: 0.06791355115160949
Accuracy: 98.48%, F1 Score: 0.98
Training with LR=0.001, Batch Size=128, Optimizer=SGD
Epoch 1/25, Loss: 2.302672129450068
Epoch 2/25, Loss: 2.301206218662547
Epoch 3/25, Loss: 2.2997813524722037
Epoch 4/25, Loss: 2.2984112874785465
Epoch 5/25, Loss: 2.296954906825572
Epoch 6/25, Loss: 2.2954824128384783
Epoch 7/25, Loss: 2.293757662335947
Epoch 8/25, Loss: 2.2917933636891052
Epoch 9/25, Loss: 2.2896258841191273
Epoch 10/25, Loss: 2.2868151029289914
Epoch 11/25, Loss: 2.2833589185783856
Epoch 12/25, Loss: 2.2791449967732054
Epoch 13/25, Loss: 2.273430465889384
Epoch 14/25, Loss: 2.2660256296332713
Epoch 15/25, Loss: 2.2556272869679463
Epoch 16/25, Loss: 2.24092428770655
Epoch 17/25, Loss: 2.217526837960998
Epoch 18/25, Loss: 2.18051575445163
Epoch 19/25, Loss: 2.1157435420225426
Epoch 20/25, Lo

KeyboardInterrupt: 

In [4]:
results = sorted(results, key=lambda x: x['f1_score'], reverse=True)
print("Top Performances:")
for result in results[:50]:
    print(result)


Top Performances:
{'lr': 0.001, 'batch_size': 64, 'optimizer': 'RMSprop', 'accuracy': 0.9911, 'f1_score': 0.9910972658442102}
{'lr': 0.001, 'batch_size': 32, 'optimizer': 'Adam', 'accuracy': 0.991, 'f1_score': 0.9909983860845659}
{'lr': 0.001, 'batch_size': 64, 'optimizer': 'Adam', 'accuracy': 0.9903, 'f1_score': 0.9903058128715826}
{'lr': 0.001, 'batch_size': 128, 'optimizer': 'RMSprop', 'accuracy': 0.9898, 'f1_score': 0.9898013697216468}
{'lr': 0.0001, 'batch_size': 64, 'optimizer': 'RMSprop', 'accuracy': 0.9895, 'f1_score': 0.9894985491587075}
{'lr': 0.01, 'batch_size': 32, 'optimizer': 'SGD', 'accuracy': 0.9886, 'f1_score': 0.9885963798807932}
{'lr': 0.0001, 'batch_size': 32, 'optimizer': 'RMSprop', 'accuracy': 0.9885, 'f1_score': 0.988504158512488}
{'lr': 0.0001, 'batch_size': 64, 'optimizer': 'Adam', 'accuracy': 0.9882, 'f1_score': 0.988194569538461}
{'lr': 0.001, 'batch_size': 32, 'optimizer': 'RMSprop', 'accuracy': 0.9878, 'f1_score': 0.9878017757449852}
{'lr': 0.001, 'batch_si

In [2]:
from torchvision import transforms


# Function to Train and Evaluate with Hyperparameter Logging
def train_and_evaluate(model, train_loader, test_loader, lr, batch_size, optimizer_type):
    print(f"Training with LR={lr}, Batch Size={batch_size}, Optimizer={optimizer_type.__name__}")
    
    # Reinitialize Model
    model = LeNet().to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optimizer_type(model.parameters(), lr=lr)
    
    # Training Loop
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader)}")
    
    # Evaluation
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    accuracy = sum([1 for true, pred in zip(all_labels, all_preds) if true == pred]) / len(all_labels)
    f1 = f1_score(all_labels, all_preds, average='weighted')

    print(f"Accuracy: {accuracy*100:.2f}%, F1 Score: {f1:.2f}")
    return accuracy, f1

# Augmentation for Training Data
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),       # Randomly flip horizontally
    transforms.RandomRotation(10),           # Random rotation by ±10 degrees
    transforms.RandomResizedCrop(28, scale=(0.8, 1.0)),  # Random cropping and scaling
    transforms.ColorJitter(brightness=0.2, contrast=0.2),  # Adjust brightness and contrast
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))     # Normalize
])

# Standard Transformation for Test Data
test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Load Datasets with Transformations
train_dataset = datasets.MNIST(root='./data', train=True, transform=train_transform, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=test_transform, download=True)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

# Reuse the Model, Training, and Evaluation Functions
accuracy, f1 = train_and_evaluate(LeNet(), train_loader, test_loader, lr=0.001, batch_size=64, optimizer_type=optim.RMSprop)

print(f"Accuracy with Augmentation: {accuracy*100:.2f}%, F1 Score with Augmentation: {f1:.2f}")


Training with LR=0.001, Batch Size=64, Optimizer=RMSprop
Epoch 1/25, Loss: 0.46860229279567944
Epoch 2/25, Loss: 0.20873892185157106
Epoch 3/25, Loss: 0.16358606952991187
Epoch 4/25, Loss: 0.14018172402776827
Epoch 5/25, Loss: 0.12541642670742056
Epoch 6/25, Loss: 0.11454202352103585
Epoch 7/25, Loss: 0.10413099325367256
Epoch 8/25, Loss: 0.09989011221414389
Epoch 9/25, Loss: 0.09789784025273789
Epoch 10/25, Loss: 0.09300048935881405
Epoch 11/25, Loss: 0.09003821961598586
Epoch 12/25, Loss: 0.08647762525003594
Epoch 13/25, Loss: 0.08263417909235589
Epoch 14/25, Loss: 0.0822591709212831
Epoch 15/25, Loss: 0.081339598181588
Epoch 16/25, Loss: 0.07761746563818127
Epoch 17/25, Loss: 0.07589666746325417
Epoch 18/25, Loss: 0.075303801169335
Epoch 19/25, Loss: 0.07493328791348014
Epoch 20/25, Loss: 0.07349263487678112
Epoch 21/25, Loss: 0.0732378926098859
Epoch 22/25, Loss: 0.07171732661955213
Epoch 23/25, Loss: 0.0703032289944174
Epoch 24/25, Loss: 0.06846904435179142
Epoch 25/25, Loss: 0.06

In [7]:
import time
import torch

# Set Model to Evaluation Mode
model.eval()

# Create a Single Input Sample (Shape for MNIST: [1, 1, 28, 28])
dummy_input = torch.randn(1, 1, 28, 28).to(device)

# Measure Latency
num_iterations = 100
latency_times = []

with torch.no_grad():
    for _ in range(num_iterations):
        start_time = time.time()
        _ = model(dummy_input)  # Forward pass
        end_time = time.time()
        latency_times.append(end_time - start_time)

# Calculate Average Latency
average_latency = sum(latency_times) / num_iterations
print(f"Average Latency per Sample: {average_latency*1000:.4f} ms")


Average Latency per Sample: 0.7271 ms


In [16]:
/////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

In [19]:
import torch

# Export the model to ONNX format
dummy_input = torch.randn(1, 1, 28, 28).to(device)  # Adjust shape for your input
torch.onnx.export(
    model, 
    dummy_input, 
    "lenet_model.onnx", 
    input_names=["input"], 
    output_names=["output"], 
    opset_version=11
)
print("Model exported to ONNX format: lenet_model.onnx")


Model exported to ONNX format: lenet_model.onnx


In [27]:
print(trt.__version__)


10.7.0
